In [ ]:
import h2o
from h2o.automl import H2OAutoML
import pandas as pd
import numpy as np

# Инициализация H2O
h2o.init()

# Загрузка данных (пример с датасетом по классификации)
data = pd.read_csv('dataset.csv')
# Конвертация в H2O фрейм
h2o_data = h2o.H2OFrame(data)

# Определение переменных
y = 'target'  # Целевая переменная
x = list(h2o_data.columns)
x.remove(y)  # Все колонки кроме целевой

# Разделение на тренировочную и тестовую выборки
train, test = h2o_data.split_frame(ratios=[0.8], seed=42)

# Настройка и запуск AutoML
aml = H2OAutoML(
    max_runtime_secs=3600,    # Максимальное время работы (1 час)
    max_models=20,            # Максимальное количество моделей
    seed=42,                  # Фиксация seed для воспроизводимости
    sort_metric='AUC',        # Метрика для сортировки моделей
    balance_classes=True,     # Балансировка классов
    exclude_algos=['DeepLearning']  # Можно исключить определенные алгоритмы
)

# Запуск AutoML
aml.train(x=x, y=y, training_frame=train, validation_frame=test)

# Получение лидерборда моделей
lb = aml.leaderboard
print(lb.head(10))

# Лучшая модель
best_model = aml.leader

# Предсказание
preds = best_model.predict(test)
print(preds.head())

# Оценка модели
performance = best_model.model_performance(test)
print(performance)

# Сохранение модели
model_path = h2o.save_model(model=best_model, path="./models", force=True)
print(f"Модель сохранена по пути: {model_path}")

# Для регрессии можно использовать другие метрики
if aml.leader.type == 'regressor':
    print(f"RMSE: {performance.rmse()}")
    print(f"R2: {performance.r2()}")
else:
    # Для классификации
    print(f"AUC: {performance.auc()}")
    print(f"F1: {performance.F1()}")
    
# Закрытие H2O
h2o.shutdown()

In [ ]:
def make_predictions(model, new_data):
    h2o_newdata = h2o.H2OFrame(new_data)
    predictions = model.predict(h2o_newdata)
    return predictions.as_data_frame()

In [ ]:
aml = H2OAutoML(
    max_models=20,
    seed=42,
    nfolds=5,  # 5-кратная кросс-валидация
    fold_column='fold_column',  # Можно указать колонку с фолдами
    keep_cross_validation_predictions=True,  # Сохранять предсказания кросс-валидации
    keep_cross_validation_models=True       # Сохранять модели кросс-валидации
) # cross-val

In [ ]:
aml = H2OAutoML(
    max_models=20,
    seed=42,
    exclude_algos=None,
    include_algos=['GBM', 'RF', 'XGBoost', 'GLM'],
    metalearner_algorithm='AUTO'  # Можно выбрать 'GLM', 'GBM', 'DRF' и др.
) # ensemble

In [ ]:
aml = H2OAutoML(
    max_models=20,
    seed=42,
    balance_classes=True,
    class_sampling_factors=[0.2, 1.0],  # Соотношение классов
    max_after_balance_size=3.0  # Максимальный размер после балансировки
) # disbalance